# SPARQL queries for SAM
In this notebook are present all the SPARQL queries devised in order to answer the competency questions and test the model's compliance. 

In [1]:
# Uncomment this section if the dependencies are needed and you want to install
# them globally or on the current environment
#! pip install rdflib pandas jinja2

In [2]:
# In this block
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import NamespaceManager
import pandas as pd

def setting_the_graph():
    g = Graph()
    g.parse(location="dataset.ttl")

    datasam = Namespace("https://github.com/falaimo99/sam/data/")
    sam = Namespace("https://purl.org/samcore#")
    wdt = Namespace("https://www.wikidata.org/wiki/")
    wdp = Namespace("https://www.wikidata.org/wiki/Property:")
    cwrc = Namespace("https://sparql.cwrc.ca/ontologies/cwrc.html#")
    rel = Namespace("https://vocab.org/relationship/#")

    g.bind("datasam", datasam)
    g.bind("sam", sam)
    g.bind("wdt", wdt)
    g.bind("wdp", wdp)
    g.bind("cwrc", cwrc)
    g.bind("rel", rel)

    return g

# Similarly to the main script we first set the graph and then start a 
# NamespaceManager object to improve visualization
g = setting_the_graph()
nm = NamespaceManager(g)

# test query
q = """
SELECT ?subject ?predicate ?object WHERE {?subject ?predicate ?object} LIMIT 10
"""

#this function automatizes the query visualization through pandas returning a 
# Dataframe that can be easily exported and used for analysis purposes
def print_query(g, q):
    data = {}

    col_counter = 0
    columns = []
    query_result = g.query(q)
    query_result_len = query_result.vars.__len__()
    for col in range(query_result_len):
        col_name = nm.normalizeUri(query_result.vars[col_counter])[1:]
        columns.append(col_name)
        data[col_name] = []
        col_counter += 1

    col_counter = 0
    for r in range(query_result_len):
        for r in query_result:
            # This conditional checks for the class of each row element
            # if not checked normalizeUri would rise an error
            if type(r[col_counter]) != Literal:
                data[columns[col_counter]].append(nm.normalizeUri(r[col_counter]))
            else:
                data[columns[col_counter]].append(r[col_counter])
        col_counter += 1
    
    df = pd.DataFrame(data, columns=columns)
    pd.set_option("display.max_colwidth", None)
    return df

print_query(g, q)

,subject,predicate,object
0,datasam:Novella_V_Giornata_II_Decameron_Boccaccio_andreuccioDiPietro,wdp:P2561,datasam:Novella_V_Giornata_II_Decameron_Boccaccio_andreuccioDiPietro_name
1,datasam:Belfagor_Arcidiavolo_Machiavelli_figliuolaDiReLodovicoVII,rdf:type,wdt:Q95074
2,datasam:Belfagor_Arcidiavolo_Machiavelli_scene10,wdp:P710,datasam:Belfagor_Arcidiavolo_Machiavelli_ReLodovicoVII
3,datasam:Petrosinella_CuntodeliCunti_Basile_Scene4,rdf:type,sam:Scene
4,datasam:Belfagor_Arcidiavolo_Machiavelli_scene12,wdp:P710,datasam:Belfagor_Arcidiavolo_Machiavelli_accompagnatoriDellaFigliuoladiLodovico
5,datasam:Petrosinella_CuntodeliCunti_Basile_orca,sam:narratorName,orca
6,datasam:Novella_del_Grasso_Legnaiuolo_anonimo_Scene9_excerpt,rdf:type,wdt:Q1385610
7,datasam:Petrosinella_CuntodeliCunti_Basile_Scene1,sam:hasSetting,datasam:Petrosinella_CuntodeliCunti_Basile_giardinoOrca
8,datasam:trope_TheProtagonist,rdf:type,sam:PeriodicTableTrope
9,datasam:Belfagor_Arcidiavolo_Machiavelli_CharacterList,wdp:P527,datasam:Belfagor_Arcidiavolo_Machiavelli_figliuoladiCarlo


In [3]:
#mini function for visualization purposes
def visualizer(df, columns, placeholders):

    # Define your maximum desired length and the placeholder
    max_length = 15  # Adjust this value as needed

    # Create a copy of the DataFrame for visualization so you don't modify the original data
    df_visual = df.copy()

    counter = 0
    for col in columns:
        df_visual[columns[counter]] = df_visual[columns[counter]].apply(
            lambda x: placeholders[counter] if isinstance(x, str) and len(x) > max_length else x
        )
        counter += 1

    return df_visual

## Character queries
### Meta
- Query 1. How many characters are present in a given story?
### Diegetic
- Query 2. Does he/she/they belong to a distinct group (organization, ethnic group, otherworldly beings, nation)?  
- Query 3. What’s their name? Do they have any aliases? 
- Query 4. The aliases or name are assigned by which other character or group of characters?
- Query 5. Are they part of a group of characters?
- Query 6. What’s their gender?
- Query 7. What’s their occupation or diegetic role?
- Query 8. Has the character gone through a change?
 
### Narratological:
- Query 9. What is a usable definition of a character in this context?
- Query 10. What narratological role does the character assume? 
- Query 11. Are they part of a recognizable pattern? Possibly coming from The Periodic Table of Storytelling (Hero’s journey, Five man band, etc.)
- Query 12. Which style choices are made when referring to the character? 



In [4]:
from modules.queries import character_queries

# Change options according to your requested query, Character, and Story
options = {
    # all URIs should be normalized in the form prefix:suffix
    # not available at the moment, if desired you can change queries and URIs
    # in "/sam/script/modules/queries.py"

    # "character_URI": "",
    # "story_URI": "",
    # 
    "selected_query_number" : "1"
}

df = print_query(g, character_queries[f"query_{options["selected_query_number"]}"])
df


,story,character_number
0,datasam:Novella_V_Giornata_II_Decameron_Boccaccio,10


## Scene queries
### Diegetic:
- Query 1. Who takes part in the scene?
- Query 2. Where does the scene take place?
- Query 3. When does the scene take place according to the diegetic time? __(not in dataset)__

### Narratological:
- Query 4. What is a fitting definition for scenes? How do we separate them? 
- Query 5. Can I find my way back in the reference media starting from a scene?
- Query 6. Can scenes track state changes in the characters and the world?
- Query 7. What's the content of the scene or a contained trope? __(not in dataset)__
- Query 8. Does a scene belong to a series of scenes, creating a subset, or arc? __(not in dataset)__
- Query 9. Does this series of scenes fit into a known narratological pattern?

In [5]:
from modules.queries import scene_queries

# Change options according to your requested query, Character, and Story
options = {
    # all URIs should be normalized in the form prefix:suffix
    # not available at the moment, if desired you can change queries and URIs
    # in "/sam/script/modules/queries.py"
    # "scene_URI": "",
    # "story_URI": "",
    #
    "selected_query_number": "5"
}


print_query(g, scene_queries[f"query_{options["selected_query_number"]}"])

,scene,scene_excerpt,scene_source
0,datasam:Novella_V_Giornata_II_Decameron_Boccaccio_Scene6,datasam:Novella_V_Giornata_II_Decameron_Boccaccio_Scene6_excerpt,https://it.wikisource.org/w/index.php?title=Decameron/Giornata_seconda/Novella_quinta&oldid=1999922


## Trope queries
Narratological:
- Query 1. Definition of a trope that excludes all the possible figurative speech or techniques involving the embodying media, but flexible enough to include various sources;
- Query 2. How can the trope be described?



In [6]:
from modules.queries import trope_queries

# Change options according to your requested query, Character, and Story
options = {
    # all URIs should be normalized in the form prefix:suffix
    # not available at the moment, if desired you can change queries and URIs
    # in "/sam/script/modules/queries.py"
    # "scene_URI": "",
    # "story_URI": "",
    #
    "selected_query_number": "2"
}


print_query(g, trope_queries[f"query_{options["selected_query_number"]}"])

,trope,trope_source
0,datasam:trope_KnightInShiningArmor,https://tvtropes.org/pmwiki/pmwiki.php/Main/KnightInShiningArmor


## Setting questions
Diegetic:
- Query 1.What place is the setting describing?
- Query 2.Which are its coordinates?
- Query 3.If the place isn’t real what’s its diegetic descriptor?
- Query 4.What is the time for this setting?
- Query 5.Which is the standard datetime data for the time?
- Query 6.If the time is just hinted at, what’s its diegetic descriptor?

Narratological:
- Query 7. What is a fitting definition for Setting?


In [7]:
setting_suggested = {
    "suggested_query1": "datasam:Novella_del_Grasso_Legnaiuolo_anonimo_piazzadiSanGiovanni",
    "suggested_query2": "datasam:Novella_del_Grasso_Legnaiuolo_anonimo_piazzadiSanGiovanni",
    "suggested_query3": "datasam:Novella_del_Grasso_Legnaiuolo_anonimo_piazzadiSanGiovanni",
    "suggested_query4": "datasam:Novella_del_Grasso_Legnaiuolo_anonimo_piazzadiSanGiovanni",
    "suggested_query5": "datasam:Novella_del_Grasso_Legnaiuolo_anonimo_piazzadiSanGiovanni",
    "suggested_query6": "datasam:Novella_del_Grasso_Legnaiuolo_anonimo_piazzadiSanGiovanni",
    "suggested_query7": "datasam:Novella_del_Grasso_Legnaiuolo_anonimo_piazzadiSanGiovanni",
}

setting_queries = {
    "query_1": """
    SELECT ?place ?coordinates ?place_descriptor WHERE{
        % s sam:hasPlace ?place .
        ?place wdp:P625 ?coordinates .
        ?place sam:placeDescriptor ?place_descriptor .
    }
    """ %(setting_suggested['suggested_query1'])
    ,
    "query_2": """
    SELECT ?trope ?trope_source WHERE{
        % s sam:tropeURI ?trope_source
    }
"""% (setting_suggested['suggested_query1'])
}

# Change options according to your requested query, Character, and Story
options = {
    # all URIs should be normalized in the form prefix:suffix
    # not available at the moment, if desired you can change queries and URIs
    # in "/sam/script/modules/queries.py"
    # "scene_URI": "",
    # "story_URI": "",
    #
    "selected_query_number": "1"
}


df = print_query(g, setting_queries[f"query_{options["selected_query_number"]}"])

visualizer(df, ["place"], ["*piazzadiSanGiovanni"])

,place,coordinates,place_descriptor
0,*piazzadiSanGiovanni,<https://geohack.toolforge.org/geohack.php?params=43.772955555556_N_11.254755555556_E_globe:earth&language=en>,"Piazza San Giovanni, Firenze"


In [8]:
q = """
    SELECT ?character ?relationship ?relationship_target WHERE {
    BIND(uri(datasam:novellino_II_anonimo_prestoGiovanni) AS ?character) .
    BIND(uri(datasam:novellino_II_anonimo_federigoImperatore) AS ?relationship_target) .
    datasam:novellino_II_anonimo_prestoGiovanni ?relationship datasam:novellino_II_anonimo_federigoImperadore .
    FILTER(isUri(?relationship) && STRSTARTS(STR(?relationship), STR(rel:))) .
}
"""
df = print_query(g, q)
visualizer(df, ['character', 'relationship_target'], ["*prestoGiovanni", "*federigoImperatore"])

,character,relationship,relationship_target
0,*prestoGiovanni,rel:knowsByReputation,*federigoImperatore
